In [6]:
import pandas as pd
from sklearn import tree
from sklearn import metrics
import graphviz 

In [2]:
df_match = pd.read_csv('../Project1/dataset.csv')
after_match_attributes = ['home_team_goal', 'away_team_goal', 'possession_home', 'shoton_home', 'shoton_away', 
                      'shotoff_home', 'shotoff_away', 'corner_home', 'corner_away', 'cross_home', 'cross_away', 
                      'foulcommit_home', 'foulcommit_away', 'rcard_home', 'rcard_away', 'ycard_home', 'ycard_away', 
                      'throwin_home', 'throwin_away']
training = df_match.sample(frac = 0.9)
training_args = training.drop(after_match_attributes, axis=1)
test = df_match[~df_match.isin(training)].dropna()
test_args = test.drop(after_match_attributes, axis=1)

In [3]:
home_win = training['home_team_goal'] > training['away_team_goal']
clf = tree.DecisionTreeClassifier()
clf.fit(training_args, home_win)
predictions = clf.predict(test_args)
actual = test['home_team_goal'] > test['away_team_goal']
metrics.confusion_matrix(actual, predictions)

array([[77, 50],
       [47, 65]])

In [10]:
dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("home_win")

'home_win.pdf'